<a href="https://colab.research.google.com/github/TimkaThymineTimiryazevskaya/sf_data_science/blob/main/HW_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Используя информацию по мидиям Петербурга и Магадана, которые представлены в виде двух массивов petersburg и magadan в разделе ANOVA тест, проверьте данные на нормальность и на наличие корреляции. Для этого объедините 2 массива в DataFrame. Обоснуйте выбор теста на корреляцию. Сделайте вывод по гипотезе.
 
 
Также ответьте на вопрос:
Какова разница в среднем размере мидии в зависимости от города-производителя. Обоснуйте выбор теста. Сделайте вывод по гипотезе.

1. Объединяем массивы в единый DataFrame

In [16]:
import pandas as pd
from scipy.stats import shapiro
from scipy.stats import normaltest
from scipy.stats import ttest_ind

In [17]:
data_p = pd.DataFrame({'petersburg':[0.0974, 0.1352, 0.0817, 0.1016, 0.0968, 0.1064, 0.105]})
data_m = pd.DataFrame({'magadan': [0.1033, 0.0915, 0.0781, 0.0685, 0.0677, 0.0697, 0.0764,0.0689]})
data = pd.concat([data_p,data_m], axis =1)
data

,petersburg,magadan
0,0.0974,0.1033
1,0.1352,0.0915
2,0.0817,0.0781
3,0.1016,0.0685
4,0.0968,0.0677
5,0.1064,0.0697
6,0.1050,0.0764
7,NaN,0.0689


Обращаем внимание, что количество данных для стобца "petersburg" меньше количества другого. Поэтому заменим недостающее значение на среднее по столбцу.

In [18]:
data['petersburg'] = data['petersburg'].fillna(data['petersburg'].mean())

In [19]:
data

,petersburg,magadan
0,0.097400,0.1033
1,0.135200,0.0915
2,0.081700,0.0781
3,0.101600,0.0685
4,0.096800,0.0677
5,0.106400,0.0697
6,0.105000,0.0764
7,0.103443,0.0689


2. Проверим на нормальность. Проверку осуществим с помощью критерия Шапиро-Уилка (выборка небольшая)

In [20]:
H0 = 'Данные распределены нормально'
Ha = 'Данные не распределены нормально (мы отвергаем H0)'
_, p = shapiro(data)
print('p=%.3f' % p)
alpha = 0.05
if p > alpha:
	print(H0)
else:
	print(Ha)

p=0.100
Данные распределены нормально


Дополнительно проверим на нормальность с помощью теста Д’Агостино

In [21]:
_, p = normaltest(data)
#normaltest возвращает двустороннюю вероятность для проверки гипотезы

print('p=%.3f' % p[0])

# Интерпретация 

if p[0] > alpha/2:
	print(H0)
else:
	print(Ha)

p=0.042
Данные распределены нормально


/usr/local/lib/python3.7/dist-packages/scipy/stats/stats.py:1542: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=8
  "anyway, n=%i" % int(n))


Вывод: Данные распределены нормально, что показано в результате проверки по двум различным тестам

3. Проведем тест на корреляцию. Данные распределены нормально, поэтому используем корреляцию Пирсона.


In [22]:
data.corr()

,petersburg,magadan
petersburg,1.000000,0.244558
magadan,0.244558,1.000000


In [23]:
from scipy.stats import pearsonr
corr, p = pearsonr(data['petersburg'], data['magadan'])
print(corr, p)

0.24455753997284532 0.5594098062056255


In [24]:
if p>alpha:
    print(f"{p} > {alpha}. Мы не можем отвергнуть нулевую гипотезу об отсутсвии зависимости между переменными.")
else:
    print(f"{p} <= {alpha}. Мы отвергаем нулевую гипотезу об отсутсвии зависимости между переменными.")

0.5594098062056255 > 0.05. Мы не можем отвергнуть нулевую гипотезу об отсутсвии зависимости между переменными.


Вывод: Мы не можем отвергнуть нулевую гипотезу об отсутсвии зависимости между переменными.

4. Проведем сравнение выборок. Зависимая переменная (размер мидий) является количественной. Группы происходят из разных совокупностей, количество данных меньше 30. Следовательно, мы используем независимый T-тест.

In [25]:
H0 = 'Нет значимой разницы между размерами мидий в разных городах.'
Ha = 'Есть значимая разница между размерами мидий в разных городах.'
def t_test(df):
    print('\n' + "*** Результаты независимого T-теста ***")
    test_results = ttest_ind(data['petersburg'], data['magadan'], equal_var=True)

    p = round(test_results[1],4)

    if p>alpha:
        print(f"{p} > {alpha}. Мы не можем отвергнуть нулевую гипотезу. {H0}")
    else:
        print(f"{p} <= {alpha}. Мы отвергаем нулевую гипотезу. {Ha}")

t_test(data)


*** Результаты независимого T-теста ***
0.0027 <= 0.05. Мы отвергаем нулевую гипотезу. Есть значимая разница между размерами мидий в разных городах.


In [26]:
a = round((data['petersburg'].mean()-data['magadan'].mean())/data['magadan'].mean(),2)*100
print(a)

33.0


Вывод: Мы отвергаем нулевую гипотезу. Есть значимая разница между размерами мидий в разных города, составляющая в среднем 33% от размера мидий в Магадане.